In [52]:
import pandas as pd

df= pd.read_csv('data/Bitcoin_BTCUSDT.csv')
df.head()

,timestamp,open,high,low,close,volume
0,2018-01-01 00:01:00,13707.91,13707.91,13666.11,13694.92,2.113
1,2018-01-01 00:02:00,13682.00,13694.94,13680.00,13680.00,1.347
2,2018-01-01 00:03:00,13679.98,13679.98,13601.00,13645.99,11.586
3,2018-01-01 00:04:00,13645.98,13646.00,13576.28,13600.00,15.730
4,2018-01-01 00:05:00,13600.00,13600.00,13554.44,13568.00,6.395


In [53]:
df.shape

(2309065, 6)

In [54]:
import pandas as pd

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

In [55]:
features = ['open', 'high', 'low', 'close', 'volume']
data = df[features]

In [56]:
df.head()

,timestamp,open,high,low,close,volume
0,2018-01-01 00:01:00,13707.91,13707.91,13666.11,13694.92,2.113
1,2018-01-01 00:02:00,13682.00,13694.94,13680.00,13680.00,1.347
2,2018-01-01 00:03:00,13679.98,13679.98,13601.00,13645.99,11.586
3,2018-01-01 00:04:00,13645.98,13646.00,13576.28,13600.00,15.730
4,2018-01-01 00:05:00,13600.00,13600.00,13554.44,13568.00,6.395


In [57]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [58]:
import numpy as np

def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len, 3])  # close price
    return np.array(X), np.array(y)

In [59]:
SEQ_LEN = 30
X, y = create_sequences(data_scaled, SEQ_LEN)

In [60]:
# Downsample to every 60th sample for faster training on CPU
split = int(0.8 * len(X))
X_train, X_test = X[:split:60], X[split::60]
y_train, y_test = y[:split:60], y[split::60]

print(f"Downsampled X_train shape: {X_train.shape}", f"X_test shape: {X_test.shape}", sep="\n")

Downsampled X_train shape: (30788, 30, 5)
X_test shape: (7697, 30, 5)


In [61]:
import torch
from torch.utils.data import TensorDataset, DataLoader


EPOCHS = 2
BATCH_SIZE = 512


train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32),
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [62]:
import torch
import torch.nn as nn

class CryptoGRU(nn.Module):
    def __init__(self, input_size, hidden_size=32):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        return self.fc(out)

In [63]:
model = CryptoGRU(input_size=5)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [64]:
from tqdm import tqdm


for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for xb, yb in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False):
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs.squeeze(), yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss:.6f}")

Epoch 1/2, Loss: 0.020816


Epoch 2/2, Loss: 0.000047


In [65]:
from torch.utils.data import TensorDataset, DataLoader

test_dataset = TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.float32),
)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [66]:
model.eval()
total_loss = 0.0
with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        batch_loss = criterion(preds.squeeze(), yb)
        total_loss += batch_loss.item() * xb.size(0)

mse = total_loss / len(test_loader.dataset)
print("Test MSE:", mse)

Test MSE: 7.050184639820541e-05


In [67]:
def predict_next_close(model, recent_data, scaler, seq_len=30):
    """
    recent_data: list of last rows [[open, high, low, close, volume], ...]
    """

    recent_data = np.array(recent_data)
    scaled = scaler.transform(recent_data)

    input_tensor = torch.tensor(
        scaled.reshape(1, seq_len, 5),
        dtype=torch.float32
    )

    model.eval()
    with torch.no_grad():
        pred_scaled = model(input_tensor).numpy()

    # inverse scale close price
    dummy = np.zeros((1, 5))
    dummy[0, 3] = pred_scaled[0, 0]
    predicted_close = scaler.inverse_transform(dummy)[0, 3]

    return predicted_close

In [68]:
recent_prices = data.iloc[-30:].values
predicted_price = predict_next_close(model, recent_prices, scaler)

print("Predicted Next Close Price:", predicted_price)

Predicted Next Close Price: 30936.555186908845


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
